## Running structure predictions with physics based MD Engine enabled refinment and binding scores (mm-GBSA)

Before running this notebook, please ensure you:

1. Are logged in by running `dm login EMAIL` in the terminal
2. Have a Token registered and saved on the file


!pip install deepmirror

!dm login <YOUREMAIL>

In [ ]:
import time
import zipfile

import pandas as pd
import py3Dmol

import deepmirror.api as api

### Define some helper functions

In [ ]:
def cofold(chains: list[dict], user_settings: dict) -> str:
    response = api.structure_prediction(chains, user_settings)
    task_id = response["task_id"]

    while True:
        response = api.get_structure_prediction(task_id)
        status = response["status"]
        if status == "completed":
            break
        print(
            f"{task_id}: Current status: {status} - Waiting 2 min for completion..."
        )
        time.sleep(120)

    with open(f"result-{task_id}.zip", "wb") as f:
        f.write(api.download_structure_prediction(task_id))

    return task_id


def view_results(task_id: str):
    with zipfile.ZipFile(f"result-{task_id}.zip", "r") as zf:
        cif_data = zf.read("data.cif").decode("utf-8")
    view = py3Dmol.view(width=400, height=400)
    view.addModel(cif_data, "cif")
    view.setStyle({"cartoon": {}})
    view.addStyle({"hetflag": True}, {"stick": {}})
    view.zoomTo()
    return view


def get_gbsa_scores(task_id: str):
    with zipfile.ZipFile(f"result-{task_id}.zip", "r") as zf:
        with zf.open("results/gbsa_scores.csv") as csvfile:
            gbsa_scores = pd.read_csv(csvfile)
    return gbsa_scores

# Protein + Ligand

In [ ]:
chains = [
    {
        "label": "A",
        "value": "GACAGCUGCUGUC",
        "type": "rna",
    },
    {
        "label": "B",
        "value": "GACAGCUGCUGUC",
        "type": "rna",
    },
    {
        "label": "C",
        "value": "Cc1cc(c2cc(ccc2n1)C(=N)N)Nc3cccc(c3)OC",
        "type": "ligand",
    },
]

### Initiate Structure Prediction

- Flag "minimize_predictions": Please set yes or no, Turns the minimization on or off respectively.
- Flag: "affinity": Please adjust it to the chain ID of the ligand

Please note that for MD based refinement, we leverage Quantum Mechanical calculations to refine small molecule's geometry and energy. Therefore, for larger and particularly charged molecules, the calculations can take upto 40 minutes.


In [ ]:
user_settings = {"minimize_predictions": "yes", "affinity": "C"}
task_id = cofold(chains, user_settings)

In [ ]:
view = view_results(task_id)
view

In [ ]:
gbsa_scores = get_gbsa_scores(task_id)
print("GBSA Scores of predicted structures")
gbsa_scores